## Direct location with Sentinel-3 SRAL sensor 

This notebook shows examples on direct location performed using SRAL sensor.

In [ ]:
import os
import json

import numpy as np

from asgard.core.frame import FrameId
from asgard.models.body import EarthBody
from asgard.models.time import TimeReference
from asgard.sensors.sentinel3.sral import S3SRALGeometry

from asgard_legacy_drivers.drivers.geojson_util import to_points
from asgard_legacy_drivers.drivers.sentinel_3_legacy import S3LegacyDriver

### Setup variables and configure S3SRALGeometry

In [ ]:
# these directories are required and must be set beforehand
TEST_DIR = os.environ.get("TEST_DIR", "../tests")

In [ ]:
# Read orbit data from FRO file
orbit_file = os.path.join(
    TEST_DIR,
    "resources/S3/FRO",
    "S3A_OPER_MPL_ORBRES_20220510T000000_20220520T000000_0001.EOF",
)
fro_20220510 = S3LegacyDriver.read_orbit_file(orbit_file)

# Define times of acquisition
frames = {
    "offsets": np.array([8168.024560769051 + 0.000001 * k for k in range(10)], dtype="float64"),
}

navatt_gps = np.load(os.path.join(TEST_DIR, "resources/sample_timestamps_gps.npy"))
navatt_oop = np.load(os.path.join(TEST_DIR, "resources/sample_oop.npy"))

# Note: here we use inertial coordinates for orbit PV
navatt_orb = S3LegacyDriver.read_orbit_file(os.path.join(TEST_DIR, "resources/sample_orbit_eme2000.xml"))
# Note: convertion to EOCFI convention not needed, already accounted in platform model
navatt_att = S3LegacyDriver.read_attitude_file(os.path.join(TEST_DIR, "resources/sample_attitude.xml"))
# We set a common time scale for orbit and attitude -> GPS
navatt_orb["time_ref"] = "GPS"
navatt_att["times"]["GPS"] = navatt_orb["times"]["GPS"]
navatt_att["time_ref"] = "GPS"

# Read EOP data
iers_data = S3LegacyDriver.read_iers_file(os.path.join(TEST_DIR, "resources/bulletinb-413.txt"))
time_model = TimeReference(iers_bulletin_b=iers_data)
body = EarthBody(time_reference=time_model)

# Change orbit frame from Earth Fixed to EME2000
driver = S3LegacyDriver(body)
fro_20220510_eme = driver.change_orbit_frame(orbit=fro_20220510, frame=FrameId.EME2000)

config = {
    "eop": {
        "iers_bulletin_b": iers_data,
    },
    "sat": "SENTINEL_3",
    "orbit_aux_info": {
        "orbit_state_vectors": [fro_20220510_eme],
    },
    "frame": {"times": frames},
    "navatt": {
        "orbit": navatt_orb,
        "attitude": navatt_att,
        "times": {
            "offsets": navatt_gps,
            "ref": "GPS",
        },
        "oop": navatt_oop,
    },
}

sral = S3SRALGeometry(**config)

# Define image coordinates
img_coord = np.column_stack((np.arange(10), np.arange(10)))

# Compute direct location
gnd = sral.direct_loc(img_coord)


### Visualize direct location on map 

In [ ]:
# creating the GeoJSON file of data points
live_dir = os.path.join(TEST_DIR, "outputs/live")
if not os.path.exists(live_dir):
    os.makedirs(live_dir)

path_points = os.path.join(live_dir, "sral_points.geojson")

to_points(gnd, path_points)

In [ ]:
# opening file and extracting coordinates
with open(path_points, 'r') as f:
    json_data_point = json.load(f)

coordinates = json_data_point['features'][0]['geometry']['coordinates']

In [ ]:
import folium

# initializing the locations map
m = folium.Map(location=coordinates[0], zoom_start=3)
# iterate through the coordinates and add them as circle-shaped markers to the map
for feature in coordinates:
    coord = feature[::-1]
    marker = folium.CircleMarker(location=coord, radius=5, color='blue', fill=True, fill_color='blue')
    marker.add_to(m)

# saving the resulting map
m.save('map_sral_points.html')

In [ ]:
# plot resulting map
m